# Import Packages and Dataset

In [234]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

In [235]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression

In [312]:
# import breast cancer dataset for binary classification
x,y = load_breast_cancer(return_X_y=True)
dim_x = x.shape[1]

# Utilities

In [325]:
tf_float_type = tf.float64
np_float_type = np.float64
as_tf_float = lambda x: tf.cast(x, tf_float_type)

In [326]:
def tf_logsumexp(ary, axis=1, keepdims=False):
    return tf.math.reduce_logsumexp(ary, axis=axis, keepdims=keepdims)

def tf_logmeanexp(ary, axis=1, keepdims=False):
    return tf.math.reduce_logsumexp(ary, axis=axis, keepdims=keepdims) \
        - tf.math.log(as_tf_float(ary.shape[axis]))

# Logistic Regression on Dataset

In [237]:
m = LogisticRegression(max_iter=10000).fit(x,y)
m.coef_

array([[ 1.02743284,  0.18106721, -0.2734324 ,  0.02231185, -0.18160364,
        -0.22197946, -0.54139022, -0.30176878, -0.27041253, -0.02992543,
        -0.08151345,  1.25273389,  0.09216139, -0.10652557, -0.02567079,
         0.07200057, -0.0321638 , -0.03869664, -0.03672253,  0.01478789,
         0.12643447, -0.43542492, -0.1012318 , -0.01378675, -0.36257047,
        -0.68317936, -1.42725958, -0.61444911, -0.74060591, -0.09397159]])

In [238]:
m.score(x,y)

0.9578207381370826

# Bayesian Logistic Regression on dataset

## Variatioal Bayes by KL

In [423]:
def pointwise_ELBO(loc_beta, scale_beta, beta):
    """
    Compute ELBO for n_MC samples of beta, where the shape of beta is [n_MC, dim_x].
    
    Arguments:
    loc_beta: 1-d array of size [dim_x]
    scale_beta: 1-d array of size [dim_x]
    beta: 2-d array of size [n_MC, dim_x] representing the n_MC samples of beta taken from q_beta
    
    Returns:
    elbo: n_MC MC-samples of elbo
    """
    p_beta = tfp.distributions.Normal(loc=np.zeros([dim_x]), scale=np.ones([dim_x]))
    q_beta = tfp.distributions.Normal(loc=loc_beta, scale=scale_beta)
    z = beta @ x.T# of shape [n_MC, n_train]
    p_y = tfd.Bernoulli(logits=z)
    y_ = y.reshape([1, -1])
    elbo = tf.reduce_sum( p_y.log_prob(y_), axis=1)\
            + tf.reduce_sum( p_beta.log_prob(beta), axis=1)\
            - tf.reduce_sum( q_beta.log_prob(beta), axis=1)
    return elbo

In [424]:
def ELBO(loc_beta, scale_beta, n_MC=1):
    q_beta = tfp.distributions.Normal(loc=loc_beta, scale=scale_beta)
    beta = q_beta.sample(n_MC)# of shape [n_MC, dim_x]
    elbo = tf.reduce_mean( pointwise_ELBO(loc_beta, scale_beta, beta) )
    return elbo

In [449]:
ELBO(loc, scale, n_MC=100)

<tf.Tensor: shape=(), dtype=float64, numpy=-20558.970077715592>

In [463]:
def pointwise_IWELBO(loc_beta, scale_beta, beta):
    """
    Compute ELBO for n_MC samples of beta, where the shape of beta is [n_MC, dim_x].
    
    Arguments:
    loc_beta: 1-d array of size [dim_x]
    scale_beta: 1-d array of size [dim_x]
    beta: 3-d array of size [n_MC_out, n_MC_in, dim_x] representing the n_MC samples of beta taken from q_beta
    
    Returns:
    elbo: n_MC MC-samples of elbo
    """

    p_beta = tfp.distributions.Normal(loc=np.zeros([dim_x]), scale=np.ones([dim_x]))
    q_beta = tfp.distributions.Normal(loc=loc_beta, scale=scale_beta)
    z = beta @ x.T# of shape [n_MC_out, n_MC_in, n_train]
    p_y = tfd.Bernoulli(logits=z)
    log_prob_ratio = tf.reduce_sum( p_y.log_prob(y), axis=2)\
                    + tf.reduce_sum( p_beta.log_prob(beta), axis=2)\
                    - tf.reduce_sum( q_beta.log_prob(beta), axis=2)# of shape [n_MC_out, n_MC_in]
    iwelbo = tf_logmeanexp(log_prob_ratio, axis=1)
    return iwelbo

In [464]:
def IWELBO(loc_beta, scale_beta, n_MC_out=1, n_MC_in=1):
    q_beta = tfp.distributions.Normal(loc=loc_beta, scale=scale_beta)
    beta = q_beta.sample([n_MC_out, n_MC_in])# of shape [n_MC_out, n_MC_in, dim_x]
    iwelbo = tf.reduce_mean( pointwise_IWELBO(loc_beta, scale_beta, beta) )
    return iwelbo

In [467]:
IWELBO(loc, scale, n_MC_out=100, n_MC_in=100)

<tf.Tensor: shape=(), dtype=float64, numpy=-1960.2974134941003>

In [400]:
p = np.ones([2, 1])  @ np.ones([1, 3])
q_beta = tfp.distributions.Normal(loc=p, scale=p)
beta = q_beta.sample([2, 3])# of shape [n_MC_out, n_MC_in, dim_x]
beta.shape

TensorShape([2, 3, 2, 3])

In [403]:
q_beta.log_prob(np.ones([2]))

InvalidArgumentError: Incompatible shapes: [2] vs. [2,3] [Op:RealDiv] name: Normal/log_prob/truediv/

In [314]:
def KL_term(loc_beta, scale_beta, n_MC=1):
    p_beta = tfp.distributions.Normal(loc=np.zeros([dim_x]), scale=np.ones([dim_x]))
    q_beta = tfp.distributions.Normal(loc=loc_beta, scale=scale_beta)
    beta = q_beta.sample(n_MC)# of shape [n_MC, dim_x]
    z = tf.matmul(beta, x.T)# of shape [n_MC, n_train]
    p_y = tfd.Bernoulli(logits=z)
    y_ = y.reshape([1, -1])
    pointwise_KL = tf.reduce_sum( p_beta.log_prob(beta), axis=1)\
                    - tf.reduce_sum( q_beta.log_prob(beta), axis=1)
    return tf.reduce_mean(pointwise_KL)

In [363]:
loc = tf.Variable(np.zeros([dim_x]))
invSPscale = tf.Variable(np.ones([dim_x])) # defined as inv_softplus(scale) to keep the positive constraint of scale
    
for t in range(4000):
    rho_t = 5e-6/(10+t)**0.7
    
    with tf.GradientTape() as g:
        g.watch([loc, invSPscale])
        scale = tf.math.softplus(invSPscale) 
        score = ELBO(loc, scale, 100)
    dloc, dinvSPscale = g.gradient(score, [loc, invSPscale])

    loc = loc + rho_t*dloc
    invSPscale = invSPscale + rho_t*dinvSPscale
print(score)

tf.Tensor(-19232.299635447518, shape=(), dtype=float64)


In [364]:
KL_term(loc, scale, 100)

<tf.Tensor: shape=(), dtype=float64, numpy=-7.340344955896737>

In [365]:
loc

<tf.Tensor: shape=(30,), dtype=float64, numpy=
array([ 1.18514096e-01,  2.19245705e-01,  7.23672171e-01,  9.50129894e-01,
        1.23792154e-03,  3.84262356e-04, -7.28324524e-04, -3.66230866e-04,
        2.33739626e-03,  9.24006169e-04,  6.67686981e-04,  1.73324834e-02,
        3.53407616e-03, -2.47093339e-01,  1.06908368e-04,  1.64803702e-04,
        1.63767110e-04,  7.48682636e-05,  2.88550031e-04,  4.56557099e-05,
        1.13773509e-01,  2.79959968e-01,  6.90096991e-01, -9.20269494e-01,
        1.61917785e-03,  4.18708341e-04, -9.52691694e-04, -1.59839367e-04,
        3.37947845e-03,  1.02946011e-03])>

In [366]:
scale

<tf.Tensor: shape=(30,), dtype=float64, numpy=
array([1.29307179, 1.27241473, 0.8109568 , 0.10268059, 1.31321399,
       1.31320488, 1.31321165, 1.3132146 , 1.31319809, 1.31321299,
       1.31323076, 1.31295267, 1.31266752, 1.16267937, 1.31321275,
       1.31321256, 1.31321132, 1.31321244, 1.31321532, 1.3132114 ,
       1.28732195, 1.24426838, 0.7241191 , 0.07624777, 1.31320184,
       1.31317873, 1.31320909, 1.3132238 , 1.31321991, 1.31320374])>

In [367]:
np.mean((x @ loc.numpy()>0)==y) # accuracy

0.9068541300527241

## Variational Bayes by Renyi Divergence

In [ ]:
def pointwise_D(loc_beta, scale_beta, n_MC_out=1, n_MC_in=1, gamma):
    p_beta = tfp.distributions.Normal(loc=np.zeros([dim_x]), scale=np.ones([dim_x]))
    q_beta = tfp.distributions.Normal(loc=loc_beta, scale=scale_beta)
    beta = q_beta.sample(n_MC_out)# of shape [n_MC, dim_x]
    z = tf.matmul(beta, x.T)# of shape [n_MC, n_train]
    p_y = tfd.Bernoulli(logits=z)
    y_ = y.reshape([1, -1])
    elbo = tf.reduce_sum( p_y.log_prob(y_), axis=1)\
            + tf.reduce_sum( p_beta.log_prob(beta), axis=1)\
            - tf.reduce_sum( q_beta.log_prob(beta), axis=1)
    return elbo

In [383]:
tf.Variable(np.arange(12).reshape([3,2,2])) @ np.arange(4).reshape([2,2])

<tf.Tensor: shape=(3, 2, 2), dtype=int64, numpy=
array([[[ 2,  3],
        [ 6, 11]],

       [[10, 19],
        [14, 27]],

       [[18, 35],
        [22, 43]]])>